<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/tools/Fundamental%20screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import datetime

In [2]:
today = datetime.date.today().strftime('%Y%m%d')

In [3]:
data = {
    'mktId': 'KSQ',
    'trdDd': today,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901',
}
gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_key = requests.post(gen_url, data=data)

In [4]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
r = requests.post(down_url, data={'code':gen_key.text})
r.encoding = 'EUC-KR'

In [5]:
from io import StringIO

df = pd.read_csv(StringIO(r.text))

In [6]:
df

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,4135,-95,-2.25,191332706255
1,054620,APS홀딩스,KOSDAQ,금융,17950,-50,-0.28,366076266950
2,265520,AP시스템,KOSDAQ,반도체,23800,0,0.00,363697819800
3,211270,AP위성,KOSDAQ,통신장비,14400,-950,-6.19,217185177600
4,013720,CBI,KOSDAQ,운송장비·부품,3525,-15,-0.42,186153610875
...,...,...,...,...,...,...,...,...
1514,024060,흥구석유,KOSDAQ,유통,7920,-300,-3.65,118800000000
1515,010240,흥국,KOSDAQ,기계·장비,8120,-20,-0.25,100060291520
1516,189980,흥국에프엔비,KOSDAQ,음식료·담배,3190,-20,-0.62,122454935130
1517,037440,희림,KOSDAQ,기타서비스,6380,100,1.59,88825390500


In [7]:
boundary = df.loc[df['시가총액'].between(30000000000,70000000000)]
boundary

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
10,056000,COWON,KOSDAQ,디지털컨텐츠,2860,0,0.00,56399843500
12,083660,CSA 코스믹,KOSDAQ,화학,1305,20,1.56,49504018770
19,079190,EMW,KOSDAQ,통신장비,1880,155,8.99,59754290360
20,050120,ES큐브,KOSDAQ,유통,4590,90,2.00,62259154740
23,900290,GRT,KOSDAQ,금융,951,0,0.00,64073625000
...,...,...,...,...,...,...,...,...
1479,900270,헝셩그룹,KOSDAQ,금융,485,2,0.41,38800000000
1492,011080,형지I&C,KOSDAQ,섬유·의류,1650,-5,-0.30,64276852200
1495,039610,화성밸브,KOSDAQ,기계·장비,6710,140,2.13,69853784000
1500,290270,휴네시온,KOSDAQ,소프트웨어,5910,20,0.34,56781341520


In [8]:
!pip install OpenDartReader

In [9]:
import OpenDartReader

# https://opendart.fss.or.kr/intro/main.do
api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

dart = OpenDartReader(api_key)

In [10]:
search_start = '2021-09-01'
search_end = '2021-10-21'

# search_date = datetime.date.today().strftime('%Y-%m-%d')

In [11]:
dart_df = dart.list(start=search_start, end=search_end, kind='A')
found = list(set(dart_df['stock_code']).intersection(set(boundary['종목코드'])))
out_df = dart_df[dart_df['stock_code'].isin(found)]

In [12]:
def get_recent_finstate(corp_code='', year=2021):
  quarters = ['11011', '11014', '11012', '11013']
  for q in quarters:
    # '11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서
    df = dart.finstate(corp_code, year, reprt_code=q)
    if type(df) != type(None):
      return df[df['fs_nm'] == '연결재무제표']

In [13]:
def get_incomestatement(df):
  is_df = df.loc[df['sj_nm'] == '손익계산서']
  try:
    t = is_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount', 'thstrm_add_amount', 'frmtrm_add_amount']]
    t.columns = ['published_date', 'date', 'name', 'current03', 'previous03', 'current12', 'previous12']
  except:
    t = is_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount']]
    t.columns = ['published_date', 'date', 'name', 'current03', 'previous03']
  return t

def get_is_score(is_df):
  is_score = {}
  for n in is_df['name']:
    current = str_int(is_df.loc[is_df['name'] == n, 'current03'].values[0])
    previous = str_int(is_df.loc[is_df['name'] == n, 'previous03'].values[0])
    is_score[n] = get_growth_rate(previous, current)
  return is_score

In [14]:
def get_balancesheet(df):
  bs_df = df.loc[df['sj_nm'] == '재무상태표']
  t = bs_df[['rcept_no', 'thstrm_dt','account_nm', 'thstrm_amount', 'frmtrm_amount']]
  t.columns = ['published_date', 'date', 'name', 'current03', 'previous03']
  return t

def get_bs_score(bs_df):
  bs_score = {}
  for n in bs_df['name']:
    current = str_int(bs_df.loc[bs_df['name'] == n, 'current03'].values[0])
    previous = str_int(bs_df.loc[bs_df['name'] == n, 'previous03'].values[0])
    bs_score[n] = get_growth_rate(previous, current)
  return bs_score

In [15]:
def get_growth_rate(previous, current):
  return (current-previous)/abs(previous)

def str_int(s):
  return int(s.replace(',',''))

In [16]:
score_dic = {}

for code in out_df['corp_code']:
  try:
    df = get_recent_finstate(code)
    is_df = get_incomestatement(df)
    is_score = get_is_score(is_df)

    bs_df = get_balancesheet(df)
    bs_score = get_bs_score(bs_df)

    date = out_df.loc[out_df['corp_code'] == code, 'rcept_dt'].values[0]
    name = out_df.loc[out_df['corp_code'] == code, 'corp_name'].values[0]
    print(date)
    print(name)
    print(is_score)
    print(bs_score)
    print()

    t = is_score
    t.update(bs_score)
    score_dic[name] = t
  except:
    print('ERROR', name)
    print()

20211020
씨엔플러스
{'매출액': 2.242357521632337, '영업이익': 1.585554958193215, '법인세차감전 순이익': 1.1507121309919817, '당기순이익': 1.2209454120064305}
{'유동자산': -0.045596962443209474, '비유동자산': -0.029088825319386197, '자산총계': -0.039484610510543236, '유동부채': 0.02621975144633279, '비유동부채': 0.02659511008794768, '부채총계': 0.02627243411611199, '자본금': 0.0, '이익잉여금': -0.03429013255488781, '자본총계': -0.16640265045510588}

20211014
엠에프엠코리아
{'매출액': 1.0808984177189913, '영업이익': -9.562186440540527, '법인세차감전 순이익': -8.137140470618641, '당기순이익': -5.419861270207572}
{'유동자산': 0.2986995071827873, '비유동자산': -0.09607841487709347, '자산총계': 0.27068213605710667, '유동부채': 0.12440184975020237, '비유동부채': 2.0982653936674622, '부채총계': 0.3520983665261315, '자본금': 0.06333404272089099, '이익잉여금': -1.242409724161504, '자본총계': 0.11468192232363031}

20211014
엠에프엠코리아
{'매출액': 1.0808984177189913, '영업이익': -9.562186440540527, '법인세차감전 순이익': -8.137140470618641, '당기순이익': -5.419861270207572}
{'유동자산': 0.2986995071827873, '비유동자산': -0.09607841487709347, '자산총계': 0.2706821

In [17]:
score_df = pd.DataFrame(score_dic).T
score_df

,매출액,영업이익,법인세차감전 순이익,당기순이익,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,자본금,이익잉여금,자본총계
씨엔플러스,2.242358,1.585555,1.150712,1.220945,-0.045597,-0.029089,-0.039485,0.026220,0.026595,0.026272,0.000000,-0.034290,-0.166403
엠에프엠코리아,1.080898,-9.562186,-8.137140,-5.419861,0.298700,-0.096078,0.270682,0.124402,2.098265,0.352098,0.063334,-1.242410,0.114682
이큐셀,0.594678,0.763345,0.981302,0.972132,-0.237070,-0.245817,-0.241480,-0.770124,1.411644,-0.502050,-0.542718,-0.006227,3.034134
스카이이앤엠,-0.141403,0.199751,-0.021714,-0.025982,0.281190,-0.270409,-0.103651,-0.433025,-0.075614,-0.422837,0.492820,-0.330465,0.713442
포시에스,0.092411,-0.011266,-0.204175,-0.169891,0.157506,0.079073,0.108244,0.080759,0.074166,0.080049,0.048258,0.097788,0.109553
아세아텍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
패션플랫폼,0.539893,27.566627,90.223817,114.768439,0.340536,0.130448,0.276887,0.168593,-0.168017,0.125325,0.000000,0.572256,0.324714
에스엘바이오닉스,0.097334,-18.374621,-0.586949,-0.578525,0.639343,-0.056904,0.133448,0.502452,-0.276396,0.455392,0.038316,-0.110476,0.011180
디엔에이링크,0.183512,-0.216752,0.284567,0.284567,-0.199590,0.795673,-0.102052,-0.153583,0.354319,-0.137116,0.000000,-0.055307,-0.087963


In [18]:
score_df.sort_values(by=['영업이익', '매출액'], ascending=False).head(10)

,매출액,영업이익,법인세차감전 순이익,당기순이익,유동자산,비유동자산,자산총계,유동부채,비유동부채,부채총계,자본금,이익잉여금,자본총계
패션플랫폼,0.539893,27.566627,90.223817,114.768439,0.340536,0.130448,0.276887,0.168593,-0.168017,0.125325,0.000000,0.572256,0.324714
씨엔플러스,2.242358,1.585555,1.150712,1.220945,-0.045597,-0.029089,-0.039485,0.026220,0.026595,0.026272,0.000000,-0.034290,-0.166403
이큐셀,0.594678,0.763345,0.981302,0.972132,-0.237070,-0.245817,-0.241480,-0.770124,1.411644,-0.502050,-0.542718,-0.006227,3.034134
스카이이앤엠,-0.141403,0.199751,-0.021714,-0.025982,0.281190,-0.270409,-0.103651,-0.433025,-0.075614,-0.422837,0.492820,-0.330465,0.713442
포시에스,0.092411,-0.011266,-0.204175,-0.169891,0.157506,0.079073,0.108244,0.080759,0.074166,0.080049,0.048258,0.097788,0.109553
디엔에이링크,0.183512,-0.216752,0.284567,0.284567,-0.199590,0.795673,-0.102052,-0.153583,0.354319,-0.137116,0.000000,-0.055307,-0.087963
엠에프엠코리아,1.080898,-9.562186,-8.137140,-5.419861,0.298700,-0.096078,0.270682,0.124402,2.098265,0.352098,0.063334,-1.242410,0.114682
에스엘바이오닉스,0.097334,-18.374621,-0.586949,-0.578525,0.639343,-0.056904,0.133448,0.502452,-0.276396,0.455392,0.038316,-0.110476,0.011180
아세아텍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
